In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [41]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial1')\
        .getOrCreate()

In [138]:
data = (
    spark.read.option("delimiter", ",")
    .option("header", True)
    .option("escape", '"')
    .csv("/content/drive/MyDrive/Colab Notebooks/data.csv")
)

In [176]:
schema = StructType(
    [
        StructField("id", StringType(), nullable=True),
        StructField("name", StringType(), True),
        StructField("artists", StringType(), True),
        StructField("duration_ms", DoubleType(), True),
        StructField("release_date", DateType(), True),
        StructField("year", IntegerType(), True),
        StructField("acousticness", StringType(), True),
        StructField("danceability", StringType(), True),
        StructField("energy", StringType(), True),
        StructField("instrumentalness", StringType(), True),
        StructField("liveness", StringType(), True),
        StructField("loudness", StringType(), True),
        StructField("speechiness", StringType(), True),
        StructField("tempo", StringType(), True),
        StructField("valence", StringType(), True),
        StructField("mode", StringType(), True),
        StructField("key", StringType(), True),
        StructField("popularity", DoubleType(), True),
        StructField("explicit", StringType(), True),
    ]
)


data = (
    spark.read.option("delimiter", ",")
    .option("header", True)
    .option("escape", '"')
    .schema(schema)
    .csv("/content/drive/MyDrive/Colab Notebooks/data.csv")
)
data1 = data
data2 = data


In [178]:
from pyspark.sql import functions as f

#Найти наиболее популярных артистов (средняя популярность всех произведений, в которых упомянут артист).
#Выборку производить из тех артистов, у которых общее количество упоминаний в произведениях не менее 200.
exploded_data = data.select(f.explode(f.split(col("artists"), ",")).alias("artist"), "popularity")
cleaned_data = exploded_data.withColumn("artist", f.regexp_replace(col("artist"), "[^a-zA-Z0-9 ]", "")) \
    .withColumn("artist", f.trim(col("artist")))
result = cleaned_data.groupBy("artist") \
    .agg(f.avg("popularity").alias("average_popularity"), f.count("*").alias("total_mentions")) \
    .filter(col("total_mentions") >= 200) \
    .select("artist") \
    .orderBy("average_popularity", ascending=False)
result.show(5)

#Отобразить изменение средних значений аудио характеристик от года к году.
#Такими характеристиками являются acousticness, danceability, energy, speechiness, liveness и valence .
#Произвести сортировку полученной таблицы по столбцу year по возрастанию. Средние значения округлить до 2-ух знаков после запятой.
data1.groupBy('year') \
    .agg(
        f.round(f.avg('acousticness'), 2).alias('acousticness'),
        f.round(f.avg('danceability'), 2).alias('danceability'),
        f.round(f.avg('energy'), 2).alias('energy'),
        f.round(f.avg('liveness'), 2).alias('liveness'),
        f.round(f.avg('speechiness'), 2).alias('speechiness'),
        f.round(f.avg('valence'), 2).alias('valence')
    ) \
    .sort('year') \
    .show(5)

#Найти количество произведений, выпущенных с 1951 года, в авторах которых присутствует "Sergei Rachmaninoff".
data2.filter( (col('year') >= '1951') ) \
    .filter(col("artists").like("%Sergei Rachmaninoff%")) \
    .agg({'id':'count'}) \
    .show(5)

+------------+
|      artist|
+------------+
|       Drake|
|  Kanye West|
|Taylor Swift|
|      Eminem|
|   Lil Wayne|
+------------+
only showing top 5 rows

+----+------------+------------+------+--------+-----------+-------+
|year|acousticness|danceability|energy|liveness|speechiness|valence|
+----+------------+------------+------+--------+-----------+-------+
|1921|         0.9|        0.43|  0.24|    0.22|       0.08|   0.43|
|1922|        0.94|        0.48|  0.24|    0.24|       0.12|   0.53|
|1923|        0.98|        0.57|  0.25|    0.24|        0.1|   0.62|
|1924|        0.94|        0.55|  0.35|    0.24|       0.09|   0.67|
|1925|        0.97|        0.57|  0.26|    0.24|       0.12|   0.62|
+----+------------+------------+------+--------+-----------+-------+
only showing top 5 rows

+---------+
|count(id)|
+---------+
|       46|
+---------+

